In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from datetime import date, datetime, timedelta, time
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from IPython.html.widgets import interact, fixed

In [ ]:
import scipy
from scipy.stats import norm
import scipy.optimize as op
import emcee

In [ ]:
seaborn.set_context('poster')

In [ ]:
from simtransient.models.supernova import Sn1aOpticalEnsemble

In [ ]:
Sn1aOpticalEnsemble.free_pars

In [ ]:
Sn1aOpticalEnsemble.gauss_pars

In [ ]:
Sn1aOpticalEnsemble.gauss_correlations

In [ ]:
ens = Sn1aOpticalEnsemble()

In [ ]:
print "det:", np.linalg.det(ens.gauss_cov)
ens.gauss_cov

In [ ]:
from scipy.stats import multivariate_normal
rv = multivariate_normal(mean=ens.gauss_pars.loc['mu'], cov=ens.gauss_cov)

In [ ]:
n_samples=5000
sample = rv.rvs(n_samples)
import triangle
true_params = rv.rvs()
_=triangle.corner(sample,
                 labels=ens.gauss_pars.keys(),
                 quantiles=[0.05, 0.5, 0.95],
                 truths=true_params)

In [ ]:
list(true_params)

In [ ]:
@interact(start=(0,n_samples-1,1),ntrace=(1,150,1))
def plt_traces(start=0,ntrace=50):
    stop = min(start+ntrace, n_samples)
    plt_samples = sample[start:stop]
    lcs = []
    t=np.linspace(-20, 80,1000)
    for pltpars in plt_samples:
        lcs.append(ens.evaluate(t,*pltpars,t0=0))
    seaborn.tsplot( lcs,  t, err_style="unit_traces", ls='')


In [ ]:
@interact(a=(0.9,1.4,0.1), rise_tau=(2.5,3.5,0.1), decay_tau=(12,20,0.1), t0=fixed(0))
def plot_sne(a,rise_tau,decay_tau):
    sn_instance = ens.get_curve(a,rise_tau,decay_tau)
    t=np.linspace(-rise_tau*5, decay_tau*5,1000)
#     flux = sn_instance(t)
    flux = ens.evaluate(t,a,rise_tau,decay_tau,t0=0)
    plt.plot(t,flux)
    plt.ylim(0,2)
    plt.xlim(-15,80)
    if False:
        triangle.corner(sample,
                     labels=ens.gpars.keys(),
                     truths=(a,rise_tau,decay_tau)
                       )